# Torres de Hanói

### Una ruta

In [ ]:
N = 7

**¿Qué tipo de objetos son los estados?**

Un estado es una lista de tres elementos, uno por cada poste. Cada poste se representa por una lista con los discos que contiene. Cada disco se representa por un natural $k$ donde $0 \leq k < 7$.

**¿Cuál es un estado inicial?**

In [ ]:
def estado_inicial():
    return [
        list(range(N)),
        [],
        [],
    ]

In [ ]:
estado_inicial()

**Escribe una función para generar un estado inicial aleatorio**

Se elige un poste aleatoriamente que contenga todos los discos.

In [ ]:
from random import randint

In [ ]:
def estado_inicial_aleatorio():
    estado = [[],[],[]]
    poste_lleno = list(range(N))
    estado[randint(0,2)] = poste_lleno
    return estado

In [ ]:
estado_inicial_aleatorio()

**Escribe un predicado para determinar si un estado es válido**

Debe corresponder a la forma de los estados

In [ ]:
def estado_valido(s):
    if type(s) is not list:
        return False
    # Garantizamos que s es lista
    if len(s) != 3:
        return False
    # Garantizamos que tiene 3 elementos
    for si in s:
        if type(si) is not list:
            return False
        if not all(type(x) is int and 0 <= x < N
                   for x in si):
            return False
        if si != sorted(si):
            return False
    # Garantizamos que cada poste es una lista de
    # enteros en el rango y orden adecuado
    discos = s[0] + s[1] + s[2]
    if len(discos) != N:
        return False
    # Garantizamos que hay N discos en total
    if len(set(discos)) != N:
        return False
    # Garantizamos que cada poste tiene discos distintos
    return True

In [ ]:
estado_valido(estado_inicial_aleatorio())

**¿Qué tipo de objeto son los movimientos?**

Los movimientos son tuplas $(p_i, p_f)$ que indican el índice del poste de donde se saca el disco $p_i$ y el ínidce del poste a donde se mueve este disco $p_f$.

**¿Qué movimientos se pueden realizar desde el estado inicial de la segunda pregunta?**

$(0, 1)$ y $(0, 2)$

**¿Cuál es la máxima cantidad de vecinos que puede tener un estado?**

Tres, por ejemplo, poder mover del poste 0 al 1, del 0 al 2 y del 1 al 2. Consideremos la siguiente secuencia de movimientos a partir del estado inicial de la segunda pregunta.

*inicio:*
```
[[0, 1, 2, 3, 4, 5, 6, 7],
 [],
 []]
```

*moviendo de 0 a 2:*
```
[[1, 2, 3, 4, 5, 6, 7],
 [],
 [0]]
```

*moviendo de 0 a 1:*
```
[[2, 3, 4, 5, 6, 7],
 [1],
 [0]]
```

*moviendo de 2 a 1:*
```
[[2, 3, 4, 5, 6, 7],
 [0, 1],
 []]
```

*moviendo de 0 a 2:*
```
[[3, 4, 5, 6, 7],
 [0, 1],
 [2]]
```

*moviendo de 1 a 0:*
```
[[0, 3, 4, 5, 6, 7],
 [1],
 [2]]
```

Posibles movimientos en este punto son:
- $(0,1)$
- $(0,2)$
- $(1,2)$

Otra forma de pensarlo es de la siguiente manera, representemos cada poste como vértices de una gráfica dirigida, ya que las reglas solo admiten el movimiento de un disco a un poste vacío o cuyo disco en la cima sea de mayor tamaño la relación de aristas no es reflexiva ni simétrica, por lo que la pregunta se reduce a la máxima cantidad de aristas dirigidas en la gráfica, osea $3$.

**Escribe una función para generar un vecino aleatorio sin tener que calcular todos los vecinos**

*Lema:* Para cualquier pareja de postes donde al menos uno no esté vacío existe algún movimiento válido.

In [ ]:
from random import sample

In [ ]:
def vecino_aleatorio(s):
    def tope(r):
        return N+1 if len(s[r]) == 0 else s[r][0]
    s2 = [[x for x in sr] for sr in s]
    i = None
    j = None
    while True:
        [i, j] = sample(range(3), 2)
        if len(s[i]) > 0 or len(s[j]) > 0:
            break
    if tope(i) < tope(j):
        s2[j].insert(0, s2[i].pop(0))
    else:
        s2[i].insert(0, s2[j].pop(0))
    return s2

In [ ]:
vecino_aleatorio(
    [[0, 3, 4, 5, 6, 7],
     [1],
     [2]]
)

**Escribe una función que mida qué tan bueno o malo es un estado (puede también depender del estado inicial)**

In [ ]:
def costo_iniciando_en(r):
    otros = [0, 1, 2]
    otros.pop(r)
    def costo(s):
        c = sum(d + N for d in s[r])
        [i, j] = otros
        ci = sum(d for d in s[i])
        cj = sum(d for d in s[j])
        c += min(ci,cj)
        return c
    return costo

In [ ]:
costo = costo_iniciando_en(0)

In [ ]:
costo(
    [[1],
     [0, 3, 4, 5, 6, 7],
     [2]]
)

In [ ]:
costo(
    [[0, 1],
     [3, 4, 5, 6, 7],
     [2]]
)

# SGD con pérdida logística

Para simplificar, consideramos $\phi(x) = x$

Clasificador:
$$f_\mathbf{w}(x) = \mathrm{sign}(\mathbf{w}\cdot\phi(x))$$

In [ ]:
def sign(z):
    if z >= 0:
        return +1
    return -1

def classify(w, x):
    return sign(np.dot(w, x))

Pérdida logística:

$$\mathrm{Loss}(\mathbf{w}, x, y) = \ln{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}$$

In [ ]:
import numpy as np

In [ ]:
def loss(w, x, y):
    return np.log(1+np.exp(-np.dot(w, x)*y))

Gradiente:

$$\begin{align*}
\nabla_\mathbf{w}\mathrm{Loss}(\mathbf{w}, x, y) &= \nabla_\mathbf{w} \ln{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)} \\
&= \frac{1}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times \nabla_\mathbf{w} \left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right) \\
&= \frac{1}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times\left[\nabla_\mathbf{w} 1 + \nabla_\mathbf{w}e^{-(\mathbf{w}\cdot\phi(x))y}     \right] \\
&= \frac{1}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times\left[\nabla_\mathbf{w}e^{-(\mathbf{w}\cdot\phi(x))y}     \right] \\
&= \frac{1}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times%
%
e^{-(\mathbf{w}\cdot\phi(x))y} \times
%
\nabla_\mathbf{w} \left[-(\mathbf{w}\cdot\phi(x))y\right]%
\\
&= \frac{e^{-(\mathbf{w}\cdot\phi(x))y}}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times%
%
\nabla_\mathbf{w} \left[-(\mathbf{w}\cdot\phi(x))y\right] \\
&= \frac{e^{-(\mathbf{w}\cdot\phi(x))y}}{\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times%
%
-\phi(x)y \\
&= \frac{1}{e^{(\mathbf{w}\cdot\phi(x))y}\left(1 + e^{-(\mathbf{w}\cdot\phi(x))y}\right)}\times%
%
-\phi(x)y \\
&= \frac{1}{e^{(\mathbf{w}\cdot\phi(x))y} + 1}\times%
%
-\phi(x)y \\
&= \frac{-\phi(x)y}{e^{(\mathbf{w}\cdot\phi(x))y} + 1}
\end{align*}$$

In [ ]:
def gloss(w, x, y):
    return -np.multiply(x, y) / (1 + np.exp(np.dot(w, x)*y))

Descenso de gradiente estocástico:

In [ ]:
def debug(*args):
    print(*args)
    return

def nodebug(*args):
    return

def sgd(D, w0, T, log=debug):
    n = 0
    w = w0
    for t in range(T):
        log(f"Epoch {t} -------------------------------------------------------------")
        for x, y in D:
            a = loss(w, x, y)
            b = gloss(w, x, y)
            eta = 1.0 / np.sqrt(n + 1)
            log(f"n = {n} w = {w} x = {x} y = {y} loss = {a} gloss = {b} eta = {eta}")
            w = w - eta * b
            n += 1
        log(f"! correct: {sum(classify(w, x) == y for x, y in D)}/{len(D)}")
    return w, 1.0 * sum(classify(w, x) == y for x, y in D) / len(D)

Conjunto de entrenamiento $D_1$

| x | y |
|---|---|
| -1 | -1 |
| -2 | -1 |
| +1 | +1 |
| +2 | +1 |

In [ ]:
D1 = [
    ((-1,), -1),
    ((-2,), -1),
    ((+1,), +1),
    ((+2,), +1),
]

In [ ]:
sgd(D1, (0,), 1)

Conjunto de entrenamiento $D_2$

| x | y |
|---|---|
| -1 | -1 |
| -2 | +1 |
| +1 | +1 |
| +2 | -1 |

In [ ]:
D2 = [
    ((-1,), -1),
    ((-2,), +1),
    ((+1,), +1),
    ((+2,), -1),
]

In [ ]:
sgd(D2, (0,), 1)

# Características no-lineales

Clasificador
$$f_\mathbf{w}(x) = \mathrm{sign}(\mathbf{w}\cdot\phi(x))$$

In [ ]:
D =    [((2, 2), -1),
        ((4, 1), -1),
        ((7, 2), -1),
        ((4, 3), +1),
        ((5, 3), +1),
        ((2, 4), -1),
        ((4, 6), -1),
        ((7, 5), -1),
        ((4, 4), +1),
        ((5, 4), +1)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.axes()

xc1 = []
yc1 = []
xc2 = []
yc2 = []
for (x, y), c in D:
    if c == +1:
        xc1.append(x)
        yc1.append(y)
    else:
        xc2.append(x)
        yc2.append(y)

ax.scatter(xc1, yc1, label="$+1$");
ax.scatter(xc2, yc2, label="$-1$");
ax.legend();

ax.set_xlim((0, 8));
ax.set_ylim((0, 8));

Parece que una frontera de desición circular nos permitiría resolver el problema.

¿Qué circulo elegimos?

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.axes()

xc1 = []
yc1 = []
xc2 = []
yc2 = []
for (x, y), c in D:
    if c == +1:
        xc1.append(x)
        yc1.append(y)
    else:
        xc2.append(x)
        yc2.append(y)

circ = plt.Circle((4.5, 3.5), 1, alpha=0.25, zorder=0)

ax.scatter(xc1, yc1, label="$+1$");
ax.scatter(xc2, yc2, label="$-1$");
ax.add_patch(circ);
ax.legend();

ax.set_xlim((0, 8));
ax.set_ylim((0, 8));

Círculo con centro en $(4.5, 3.5)$ y radio $1$


Ecuación general, queremos clasificar con la clase $+1$ a los puntos $x_1, x_2$ que satisfacen la siguiente desigualdad

$$(x_1-a)^2 + (x_2-b)^2 \leq r^2$$

Para una frontera de desición centrada en $(a,b)$ con radio $r$.

Desarrollamos la desigualdad:



$$\begin{align*}
(x_1-a)^2 + (x_2-b)^2 &\leq r^2 \\
%%
x_1^2 - 2ax_1 + a^2 + x_2^2 - 2bx_2 + b^2 &\leq r^2 \\
%%
x_1^2 - 2ax_1 + a^2 + x_2^2 - 2bx_2 + b^2 - r^2 &\leq 0 \\
%%
(a^2 + b^2 - r^2)1 + (-2a)x_1 + (-2b)x_2 + (1)(x_1^2+x_2^2) &\leq 0 \\
\end{align*}$$

Si un punto satisface esta desigualdad, lamentablemente, sería clasificado como $-1$ ya que tomaríamos el signo del lado izquierdo de la desigualdad. Por lo tanto, debemos ajustar la desigualdad multiplicando ambos lados por $-1$.

$$\begin{align*}
-\left[(a^2 + b^2 - r^2)1 + (-2a)x_1 + (-2b)x_2 + (1)(x_1^2+x_2^2)\right] &\geq -0 \\
%%
(r^2 - a^2 - b^2)1 + (2a)x_1 + (2b)x_2 + (-1)(x_1^2+x_2^2) &\geq 0 \\
%%
[r^2 - a^2 - b^2, 2a, 2b, -1]\cdot[1, x_1, x_2, x_1^2+x_2^2] &\geq 0 \\
%%
\mathbf{w}\cdot\phi(x) &\geq 0 \\
\end{align*}$$

In [ ]:
a = 4.5
b = 3.5
r = 1.0

w = np.array([
    r**2-a**2-b**2,
    2*a,
    2*b,
    -1,
])

def phi(x):
    return np.array([1.0, x[0], x[1], x[0]**2 + x[1]**2])

def classify(x):
    return sign(np.dot(w, phi(x)))

In [ ]:
w

Verifiquemos empíricamente si el clasificador describe el objeto geométrico deseado:

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.axes()

xc1 = []
yc1 = []
xc2 = []
yc2 = []

for x in np.linspace(0, 7, 100):
    for y in np.linspace(0, 6, 100):
        if classify((x, y)) == +1:
            xc1.append(x)
            yc1.append(y)
        else:
            xc2.append(x)
            yc2.append(y)

ax.scatter(xc1, yc1, label="$+1$", s=26, marker="s");
ax.scatter(xc2, yc2, label="$-1$", s=26, marker="s");
ax.legend();

ax.set_xlim((0, 8));
ax.set_ylim((0, 8));

In [ ]:
for x, y in D:
    assert(classify(x) == y)

Todo fino capuchino